In [1]:
import pandas as pd

In [3]:
df = pd.read_csv("spotify_millsongdata.csv")
df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [4]:
df.tail()

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [5]:
df.shape

(57650, 4)

In [6]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [7]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [8]:
df.head(10)

,artist,song,text
0,Queen Adreena,Suck,force my hand i will sign anything \r\njust c...
1,Ufo,Out In The Street,[Chorus] \r\nLady loves to hang on \r\nLady ...
2,Iggy Pop,Cold Metal,I played tag in the auto graveyard \r\nI look...
3,'n Sync,Could It Be You (bonus Track),Chorus: \r\nCould it be you? \r\nI've been s...
4,P!nk,Try Too Hard,"123... \r\nYou follow every line, you wear yo..."
5,Otis Redding,Can't Turn You Loose - Live,(Huh) I cant turn you loose now \r\nIf I do I...
6,Ella Fitzgerald,"For You, For Me For Evermore",Paradise can not refuse us \r\nNever such a h...
7,HIM,I Love You,I see it in your eyes \r\nI feel it in your t...
8,John Prine,John Prine Christmas,Pretty paper \r\nPretty ribbons of blue. \r\...
9,Bette Midler,Wind Beneath My Wings,"Ohhhh, oh, oh, oh, ohhh. \r\nIt must have bee..."


In [9]:
df['text'][0]

"force my hand i will sign anything  \r\njust cos i'm lonely  \r\nlay me down please  \r\njust do anything  \r\njust cos i'm lonely  \r\nsign your name i won't tell anyone  \r\njust cos i'm lonely  \r\nsteal my game go on take everything  \r\njust cos i'm lonely  \r\nla la la la la la la  \r\nsuck...a little bit deeper  \r\nla la la la la la la  \r\njust one for the record...suck  \r\nla la la la la la la  \r\ngive me your cash for questions  \r\nbecause i am lonely  \r\nsteal my game i won't tell anyone  \r\njust cos i'm lonely  \r\nsign your name go on take everything  \r\njust cos i'm lonely  \r\ngive me your cash for questions  \r\nbecause i am lonely\r\n\r\n"

In [10]:
df.shape

(5000, 3)

# Text Cleaning/ Text Preprocessing

In [11]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [12]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [14]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [15]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [17]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [18]:
similarity[0]

array([1.        , 0.00560345, 0.        , ..., 0.04053683, 0.02249196,
       0.00650122])

In [19]:
df[df['song'] == 'Crying Over You']

,artist,song,text


In [25]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

recommendation('Crying Over You')

In [26]:
recommendation('Crying Over You')

IndexError: index 0 is out of bounds for axis 0 with size 0

In [30]:
def recommendation(song_df, song_name, similarity):
    # Check if the song is in the DataFrame
    if song_name in song_df['song'].values:
        idx = df[df['song'] == song_name].index[0]
        
        # Calculate distances and sort them in descending order based on similarity
        distances = sorted(list(enumerate(similarity[idx])), key=lambda x: x[1], reverse=True)
        
        # Collect the top 20 similar songs
        songs = []
        for m_id in distances[1:21]:  # Start from 1 to avoid the song itself
            songs.append(df.iloc[m_id[0]]['song'])
            
        return songs
    else:
        # Return a message or empty list if the song is not found
        return "Song not found in the database."

# Example call
# Ensure your song_df DataFrame and similarity matrix are correctly defined
recommendation(df, 'Crying Over You', similarity)


'Song not found in the database.'

In [31]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))